In [24]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [25]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [26]:
# original dataset with random index
dataset_name = 'adult'
train_size = 7000
attack_test_size = 2500
test_start = 7000
data = pd.read_csv('data/adult_original_mod.csv', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 12000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [27]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
act_layer=3
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, act_layer, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  92.8428590297699 Target Test acc :  80.73333501815796


In [28]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 7000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  92.85714030265808 Shadow Test acc :  82.19047784805298
66/66 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  91.20408296585083 Shadow Test acc :  80.28571605682373
66/66 [==============================] - 0s 2ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  91.61224365234375 Shadow Test acc :  82.42856860160828
66/66 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  91.22449159622192 Shadow Test acc :  82.42856860160828
66/66 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  86.24489903450012 Shadow Test acc :  81.28571510314941
66/66 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  91.30612015724182 Shadow Test acc :  80.57143092155457
66/66 [==============================] - 0s 2

In [29]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

79/79 [==============================] - 0s 3ms/step
0
59/59 [==============================] - 0s 1ms/step
1
20/20 [==============================] - 0s 1ms/step
TP: 2456     FP: 2228     FN: 44     TN: 272
PPV: 0.5243
Advantage: 0.0912
Accuracy:  0.5456 Precision:  0.5243381725021349


In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [20]:
# synthetic dataset
train_size = 7000
attack_test_size = 2500
test_start = 7000
target_dataset = pd.read_csv('data/adult_sds.csv', na_values=["?"], header=None)
shadow_dataset = pd.read_csv('data/adultODS10K_to_25K.csv', na_values=["?"], header=None)
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_30K.csv', na_values=["?"], header=None)
attack_test_nonmembers = attack_test_nonmembers.sample(n=attack_test_size, replace=False)
attack_test_members = pd.read_csv('data/adultODS1_to_5K.csv', na_values=["?"], header=None)
attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [21]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
act_layer=3
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, act_layer, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  93.0571436882019 Target Test acc :  83.26666951179504


In [22]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 7000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  91.3673460483551 Shadow Test acc :  79.42857146263123
66/66 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  93.83673667907715 Shadow Test acc :  80.04761934280396
66/66 [==============================] - 0s 2ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  90.26530385017395 Shadow Test acc :  82.47619271278381
66/66 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  91.71428680419922 Shadow Test acc :  81.61904811859131
66/66 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  92.97959208488464 Shadow Test acc :  81.90476298332214
66/66 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  91.28571152687073 Shadow Test acc :  81.80952668190002
66/66 [==============================] - 0s 2m

In [23]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

79/79 [==============================] - 0s 3ms/step
0
58/58 [==============================] - 0s 2ms/step
1
21/21 [==============================] - 0s 1ms/step
TP: 2276     FP: 2255     FN: 224     TN: 245
PPV: 0.5023
Advantage: 0.0084
Accuracy:  0.5042 Precision:  0.5023173692341646


In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#